In [1]:
import pandas as pd
import geopandas as gpd
import gtfs_functions as gtfs
import pandas as pd,dateparser
import glob,dateparser
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import gtfs_kit as gk
import os,sys
import requests,json
import plotly.express as px
from shapely import geometry, ops
import numpy as np
from shapely.geometry import Point,LineString,MultiLineString
import math,statistics
pd.set_option('display.max_columns', None)

In [5]:
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs('2021-10-24.zip')

In [3]:
# trips

In [6]:
import plotly.graph_objects as go
def plot_me(coords,name='',fig=None,color='blue'):
    lat=[item[1] for item in coords]
    lon=[item[0] for item in coords]    
    if fig is None:
        print ('plotting')
        fig = go.Figure(go.Scattermapbox(mode = "markers+lines", lon = lon, lat = lat,    name=name,     text=name,      marker = {'size': 6,'color':color}))    
    else:
        fig.add_trace(go.Scattermapbox(mode = "markers+lines", lon = lon, lat = lat,    name=name,     text=name,      marker = {'size': 6,'color':color}))        
    fig.update_layout(
        margin ={'l':0,'t':30,'b':0,'r':0},   
        mapbox_style="open-street-map",
        mapbox = {'center': {'lon':lon[0], 'lat': lat[0]},'zoom': 15})
    return fig

    

In [7]:
def request_path(geom):
    initial='http://127.0.0.1:5000/match/v1/driving/'
    end='?overview=full&annotations=true&geometries=geojson'
    requests=[]
    l=[list(geom.coords)]
    allcoords=[item for sublist in l for item in sublist]
    maxelements=9999 #max coordinates
    overlap=2 #overlap        
    x = [allcoords[i:i+maxelements] for i in range(0, len(allcoords), maxelements-overlap)] #all requests
    for line in x:
        y=[str(item).strip('()').replace(" ", "") for item in line]
        sep=';'
        querypart=sep.join(y)
        radius="&radiuses="
        for entry in y:
            radius=radius+'25;'
        radius=radius.strip(';')
#             approaches="&approaches="
#             for entry in y:
#                 approaches=approaches+'curb;'
        #approaches=approaches.strip(';')
        requests.append(initial+querypart+end+radius)
        print(requests)
    return requests     

In [8]:
def return_new_line(reqs):
    url=reqs[0]
    r=requests.get(url)
    data=r.json()
    l=LineString(data['matchings'][0]['geometry']['coordinates'])
    return l
    
    

In [9]:
#creating matching url for paths
shapes['url']=shapes.geometry.apply(request_path)

['http://127.0.0.1:5000/match/v1/driving/-85.309722,35.052658;-85.30963,35.05266;-85.309625,35.052525;-85.30962,35.05239;-85.30962,35.05226;-85.309615,35.052155;-85.30961,35.05205;-85.309607,35.051945;-85.309605,35.05184;-85.309602,35.051735;-85.3096,35.05163;-85.309597,35.05146;-85.309595,35.05129;-85.309592,35.05112;-85.30959,35.05095;-85.309585,35.050825;-85.30958,35.0507;-85.309532,35.050703;-85.30958,35.0507;-85.309575,35.05059;-85.30957,35.05048;-85.309565,35.0503;-85.30956,35.05012;-85.309557,35.05001;-85.309555,35.0499;-85.309552,35.04979;-85.30955,35.04968;-85.309544,35.049574;-85.309537,35.049468;-85.309531,35.049361;-85.309525,35.049255;-85.309519,35.049149;-85.309512,35.049043;-85.309506,35.048936;-85.3095,35.04883;-85.309547,35.048828;-85.309563,35.048693;-85.309547,35.048527;-85.309547,35.048367;-85.309547,35.048223;-85.309547,35.04812;-85.309547,35.04797;-85.309547,35.047795;-85.309547,35.04766;-85.309547,35.047525;-85.309532,35.04737;-85.309532,35.047195;-85.309532,35.0

### map matching for stops (nearby streets)

In [147]:
def get_stop_url(geom):
    # print(df)
    # geom = df.geometry
    initial='http://127.0.0.1:5000/nearest/v1/driving/'
    end='?number=1'
    point = str(geom.x)+","+str(geom.y)
    requests2 = []
    requests2.append(initial+point+end)
    return requests2
    

In [148]:
def nearby_points(reqs):
    url=reqs[0]
    r=requests.get(url)
    data=r.json()
    l=Point(data['waypoints'][0]['location'])
    return l

In [149]:
arr=[]
arr.append(stop_times.geometry.iloc[0].x)
arr.append(stop_times.geometry.iloc[0].y)
arr

[-85.309616, 35.052191]

In [150]:
stop_times = gpd.GeoDataFrame(stop_times)
# type(k.geometry)
stop_times['url'] = stop_times.geometry.apply(get_stop_url)

In [151]:
stop_times.head(1)

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,timepoint,shape_dist_traveled,route_id,service_id,direction_id,shape_id,stop_code,stop_name,geometry,url
0,795f79b1-a-23226e03-8,2188,1,19380.0,19380.0,1,129.148935,203f5b65,1,1,1131953846,3159,Market & 4th (in front of Chili's),POINT (-85.30962 35.05219),[http://127.0.0.1:5000/nearest/v1/driving/-85....


In [152]:
data2=requests.get(stop_times['url'].iloc[0][0])

In [157]:
stops = gpd.GeoDataFrame(stops)
# type(k.geometry)
stops['url'] = stops.geometry.apply(get_stop_url)
stops['new_geom']=stops.url.apply(nearby_points)


In [158]:
stops.head(1)

,stop_id,stop_code,stop_name,geometry,stop_lat,stop_lon,url,new_geom
0,2234,3230,Highway 58 & Oakwood Dr -1-0,POINT (-85.18691 35.08955),35.08955,-85.186912,[http://127.0.0.1:5000/nearest/v1/driving/-85....,POINT (-85.186912 35.08955)


In [159]:
stop_times_upd = pd.merge(stop_times, stops[['stop_id', 'new_geom']], on=['stop_id'], how='left')

In [160]:
stop_times_upd = stop_times_upd.drop(['geometry', 'url'], axis=1)
stop_times_upd.rename(columns ={'new_geom':'geometry'}, inplace=True)

In [161]:
stops = stops.drop(['geometry', 'url'], axis=1)
stops.rename(columns ={'new_geom':'geometry'}, inplace=True)

In [162]:
type(stops.iloc[1].geometry)
stops['stop_lat'] = ''
stops['stop_lon'] = ''

In [163]:
for i, stop in stops.iterrows():
    # print((stop.geometry.x, i))
    stops.loc[i, 'stop_lat'] = stop.geometry.y
    stops.loc[i, 'stop_lon'] = stop.geometry.x
# stop_lat,stop_lon,

In [165]:
#saving matched stops
stops.drop(['geometry'], axis=1).to_csv(r'stops_matched.txt', header=True, index=None, sep=',', mode='a')

In [47]:
shapes.iloc[1]

shape_id                                           1131953808
geometry    LINESTRING (-85.268713 35.056167, -85.268713 3...
url         [http://127.0.0.1:5000/match/v1/driving/-85.26...
Name: 1, dtype: object

In [48]:
data=requests.get(shapes['url'].iloc[1][0])
r=data.json()
cords=[item['location'] for item in (r['tracepoints'])]

In [37]:
legs = pd.json_normalize(r['matchings'][0]['legs'], sep='_')

In [57]:
cords2 = [[i[1], i[0]] for i in cords] 

In [52]:
f=plot_me(cords, name='cords')
f.show()

plotting


In [39]:
cordsmatch=r['matchings'][0]['geometry']['coordinates']

In [40]:
f=plot_me(cordsmatch,'fullmatch',f,'red')

In [41]:
# np.array_equal(cords, cordsmatch)
len(cords)
len(cordsmatch)
LineString(cordsmatch).length 

0.11867134026469735

In [42]:
f.show()

In [43]:
LineString(r['matchings'][0]['geometry']['coordinates']).length

0.11867134026469735

In [44]:
legs=r['matchings'][0]['legs']

In [45]:
len(r['matchings'][0]['geometry']['coordinates'])

1128

In [46]:
legs[3]

{'steps': [],
 'summary': '',
 'weight': 0.9,
 'duration': 0.9,
 'annotation': {'metadata': {'datasource_names': ['lua profile']},
  'datasources': [0, 0],
  'weight': [0.1, 0.8],
  'nodes': [7147674228, 202596770, 574894890],
  'distance': [1.334716, 13.129756],
  'duration': [0.1, 0.8],
  'speed': [13.3, 16.4]},
 'distance': 14.5}

In [47]:
len(legs)

928

# map matching for paths/shapes

In [64]:
shapes['urllength']=shapes['url'].apply(len)

In [61]:
shapes['n_geom']=shapes.url.apply(return_new_line)

In [65]:
# shapes.to_pickle('2021-02-28-matchedShapes.pkl')

In [62]:
type(shapes)

geopandas.geodataframe.GeoDataFrame

In [66]:
shapes2=shapes.merge(trips, on='shape_id',how='left').drop(['url','urllength','trip_id','geometry','n_geom'],axis=1).drop_duplicates().reset_index().drop(['index'],axis=1)

In [77]:
shapes2=shapes2.merge(shapes,on='shape_id',how='inner')

In [133]:
shapes_actual = pd.read_csv('./gtfs_data/2022-06-10/shapes.txt', sep=',').astype(str)

In [82]:
shapes_actual.head(1)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,1131953847,35.10479,-85.1601,1


In [97]:
def linestring_to_points(line):
    return [line.coords]

shapes3['points'] = shapes3.apply(lambda l: linestring_to_points(l['geometry']),axis=1)

In [103]:
list(shapes3.iloc[0].points)

In [137]:
# shapes3 = shapes2[:2]
# import shapely.wkt as wkt
# list(shapes3.iloc[0].n_geom.coords)[0][0]
shapes3 = shapes2
temp_df = pd.DataFrame(columns = ['shape_id', 'shape_pt_lat', 'shape_pt_lon'])
# temp_df2 = pd.DataFrame(columns = ['shape_id', 'shape_pt_lat', 'shape_pt_lon'])
count=0
for idx, row in shapes3.iterrows():
    l1 = list(shapes3.iloc[idx].geometry.coords)
    l2 = list(shapes3.iloc[idx].n_geom.coords)
    for pt in range(len(l1)):
        # print(pt[1], pt[0])
        temp_df.loc[count, 'shape_id'] = shapes3.iloc[idx].shape_id
        temp_df.loc[count, 'shape_pt_lat'] = l1[pt][1]
        temp_df.loc[count, 'shape_pt_lon'] = l1[pt][0]
        temp_df.loc[count, 'shape_pt_lat_new'] = l2[pt][1]
        temp_df.loc[count, 'shape_pt_lon_new'] = l2[pt][0]

        count+=1


In [138]:
temp_df.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_lat_new,shape_pt_lon_new
0,1131953806,35.048828,-85.309547,35.048829,-85.309511
1,1131953806,35.048828,-85.309547,35.048829,-85.309511
2,1131953806,35.048828,-85.309547,35.048829,-85.309511
3,1131953806,35.048693,-85.309563,35.048695,-85.309504
4,1131953806,35.048527,-85.309547,35.048529,-85.309495


In [143]:
temp_df = temp_df.astype(str)
shapes_mod = temp_df.merge(shapes_actual, on=['shape_id', 'shape_pt_lat', 'shape_pt_lon']).drop(['shape_pt_lat', 'shape_pt_lon'], axis=1).rename(columns={'shape_pt_lat_new': 'shape_pt_lat', 'shape_pt_lon_new': 'shape_pt_lon' })

In [145]:
shapes_mod

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,1131953806,35.048829,-85.309511,1
1,1131953806,35.048829,-85.309511,2
2,1131953806,35.048829,-85.309511,3
3,1131953806,35.048829,-85.309511,1
4,1131953806,35.048829,-85.309511,2
...,...,...,...,...
16741,1131953873,35.099139,-85.280246,521
16742,1131953873,35.099139,-85.280246,522
16743,1131953873,35.099061,-85.280328,523
16744,1131953873,35.098958,-85.280403,524


In [146]:
shapes_mod.to_csv(r'shapes_matched.txt', header=True, index=None, sep=',', mode='a')

In [94]:
trips_shapes = trips.drop_duplicates(subset=['shape_id'])
trips_shapes.trip_id = trips_shapes.trip_id.apply(lambda x: int(int(x)/1000))

In [95]:
trips_shapes

,trip_id,route_id,service_id,direction_id,shape_id
0,204888,1,1,0,shp-1-04
1,204889,1,1,0,shp-1-03
36,204924,1,1,1,shp-1-66
51,204939,1,1,1,shp-1-65
72,205050,10A,1,0,shp-10A-01
...,...,...,...,...,...
809,206765,9,1,0,shp-9-03
824,206780,9,1,1,shp-9-51
825,206781,9,1,1,shp-9-52
845,206861,DTS,1,1,shp-DTS-02


In [97]:
# shapes2

In [11]:
clever_df = pd.read_csv('./20220111_spd_lat_lon.csv', index_col=[0]).sort_values('origtatripno')
clever_gdf = gpd.GeoDataFrame(clever_df)

In [41]:
clever_gdf[clever_gdf.origtatripno == 204888].geometry

0      POINT (-85.26897233333334 35.057678833333334)
119    POINT (-85.32688903808594 35.010887145996094)
120     POINT (-85.32755279541016 35.01116180419922)
121     POINT (-85.32601928710938 35.01267623901367)
122     POINT (-85.32511901855469 35.01436996459961)
                           ...                      
62        POINT (-85.3259506225586 35.0088005065918)
63        POINT (-85.3259506225586 35.0088005065918)
64        POINT (-85.3259506225586 35.0088005065918)
65        POINT (-85.3259506225586 35.0088005065918)
66        POINT (-85.3259506225586 35.0088005065918)
Name: geometry, Length: 186, dtype: object

In [39]:
# (LineString(clever_df[clever_df['origtatripno']==204888].geometry.tolist()))

AssertionError: 

In [40]:
geo_df2 = clever_df.groupby(['origtatripno'])['geometry'].apply(lambda x: LineString(x.tolist()))
geo_df2 = gpd.GeoDataFrame(geo_df2, geometry='geometry')

AssertionError: 

In [29]:
# from shapely.geometry import LineString
trips_clever = clever_df.origtatripno.unique()
clever_df2 = gpd.GeoDataFrame(columns=['trip_id', 'geometry'])
# pts= []
idx=0
for i in trips_clever:
    part_df = clever_df[clever_df['origtatripno']==i]a
    # index = trps[trps==i].index[0]
    clever_df2.loc[idx, 'trip_id'] = str(i)+"020"
    print((LineString(part_df.geometry.array)))
    clever_df2.loc[idx, 'line_geom'] = (LineString(part_df.geometry.array))
    idx+=1
    # pts = []



AssertionError: 

In [108]:
import plotly.graph_objects as go

for index in range(0, len(shapes2)):
    fig=None
    l=shapes2.n_geom.iloc[index]
    lon=list(l.coords.xy[0])
    lat=list(l.coords.xy[1])
    # print(lat)

    lold=shapes2.geometry.iloc[index]
    lon_o=list(lold.coords.xy[0])
    lat_o=list(lold.coords.xy[1])

    basename=f'{shapes2.shape_id.iloc[index]}<br>Route {shapes2.route_id.iloc[index]}<br>Service {shapes2.service_id.iloc[index]}<br>Direction {shapes2.direction_id.iloc[index]}'

    # fig = go.Figure(go.Scattermapbox(mode = "markers+lines", lon = lon, lat = lat,    name=name,     text=name,      marker = {'size': 6,'color':color}))

    fig = go.Figure(go.Scattermapbox(
        mode = "markers+lines",
        lon = lon,
        lat = lat, 
        # hoverinfosrc=["direction_id"],
        name=shapes2.shape_id.iloc[index]+'-matched',
        text=basename+'-matched',
        marker = {'size': 6,'color':'red'}))

    fig.add_trace(go.Scattermapbox(
        mode = "markers+lines",
        lon = lon_o,
        lat = lat_o,    
        # hoverinfosrc=["direction_id"],
        name=shapes2.shape_id.iloc[index]+'',
        text=basename+'',
        marker = {'size': 6,'color':'blue'}))

    fig.update_layout(
        margin ={'l':0,'t':0,'b':0,'r':0},        
        mapbox_style="open-street-map",
        mapbox = {'center': {'lon':lon[0], 'lat': lat[0]},'zoom': 15})
        # mapbox_style="streets",
        # mapbox = {'center': {'lon': -85.3097, 'lat': 35.0456},                   
        #           'accesstoken':'sk.eyJ1IjoicmlzaGF2c2VuMSIsImEiOiJjbDNtbTlhamgwNXhtM2NtZmZvc3podXE0In0.lswPpGfocU3vvRPbXajcHw',
        #     'zoom': 15})

    fig.write_html('allmatches3/'+shapes2.shape_id.iloc[index]+'.html')
    # fig.show()
    # break
